In [3]:
import requests
import zipfile
import pandas as pd
import io

In [356]:
def busca_informes_cvm(ano, mes):
  url = 'https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/'
  if ano < 2021 :
    url = url + 'HIST/'
    file_name = 'inf_diario_fi_{:02d}.zip'.format(ano)
  else :
    file_name = 'inf_diario_fi_{:02d}{:02d}.zip'.format(ano,mes)
  
  download = requests.get(url+file_name)
  download.raise_for_status()
  content = io.BytesIO(download.content)
  with zipfile.ZipFile(content) as arquivo_zip:
        # Pega o nome do primeiro arquivo dentro do ZIP
        csv_name = arquivo_zip.namelist()[0]
        
        with arquivo_zip.open(csv_name) as f:
            dados = pd.read_csv(f, sep=";", encoding='ISO-8859-1', low_memory=False)
    
  dados['TP_FUNDO_CLASSE']   =  dados['TP_FUNDO_CLASSE'].astype('string')
  dados['CNPJ_FUNDO_CLASSE'] =  dados['CNPJ_FUNDO_CLASSE'].astype('string')
  dados['ID_SUBCLASSE']      =  dados['ID_SUBCLASSE'].astype('string')
  dados['DT_COMPTC']         =  dados['DT_COMPTC'].astype('string')

  return dados

# def busca_cadastro_cvm(): 
#   url = 'https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv'
#   dados = pd.read_csv(url, sep=';', encoding='ISO-8859-1',
#   #                     usecols=['TP_FUNDO', 'CNPJ_FUNDO', 'DENOM_SOCIAL', 
#   #                              'SIT','CLASSE', 'RENTAB_FUNDO', 'CONDOM', 
#   #                              'TRIB_LPRAZO', 'PUBLICO_ALVO', 'TAXA_PERFM',
#   #                              'INF_TAXA_PERFM', 'TAXA_ADM', 'INF_TAXA_ADM', 
#   #                              'CLASSE_ANBIMA'],
#   low_memory=False)
#   # dados = dados[dados['SIT']=='EM FUNCIONAMENTO NORMAL']
#   return dados

def formatar_cnpj(cnpj):
    # Converte para string e mantém apenas dígitos
    cnpj = ''.join(filter(str.isdigit, str(cnpj)))

    # Completa com zeros à esquerda se tiver menos de 14 dígitos
    if len(cnpj) < 14:
        cnpj = cnpj.zfill(14)
    
    # Se tiver mais que 14, é erro
    if len(cnpj) != 14:
        raise ValueError("CNPJ deve conter no máximo 14 dígitos numéricos")

    # Formata
    return f"{cnpj[:2]}.{cnpj[2:5]}.{cnpj[5:8]}/{cnpj[8:12]}-{cnpj[12:]}"

def busca_cadastro_cvm():
    url = 'https://dados.cvm.gov.br/dados/FI/CAD/DADOS/registro_fundo_classe.zip'
    res = requests.get(url)
    res.raise_for_status()
    
    with zipfile.ZipFile(io.BytesIO(res.content)) as z:
        # Lendo os arquivos conforme os nomes nos metadados
        df_fundo = pd.read_csv(z.open('registro_fundo.csv'), sep=';', encoding='ISO-8859-1')
        df_fundo['CNPJ_Fundo'] = df_fundo['CNPJ_Fundo'].apply(formatar_cnpj).astype('string')
        df_classe = pd.read_csv(z.open('registro_classe.csv'), sep=';', encoding='ISO-8859-1')
        df_classe['CNPJ_Classe'] = df_classe['CNPJ_Classe'].apply(formatar_cnpj).astype('string')
        df_subclasse = pd.read_csv(z.open('registro_subclasse.csv'), sep=';', encoding='ISO-8859-1')

    # Unir Subclasse com Classe usando o ID de Registro da Classe
    mestre = pd.merge(
        df_subclasse, 
        df_classe, 
        on='ID_Registro_Classe', 
        how='outer', 
        suffixes=('', '_classe')
    )

    # Unir o resultado com o Fundo usando o ID de Registro do Fundo
    mestre = pd.merge(
        mestre, 
        df_fundo, 
        on='ID_Registro_Fundo', 
        how='outer', 
        suffixes=('', '_fundo')
    )

    return df_fundo , df_classe , df_subclasse , mestre



In [357]:
diario = busca_informes_cvm(2025,12)

In [358]:
diario[diario['CNPJ_FUNDO_CLASSE']=='48.931.854/0001-18'].head(1)

,TP_FUNDO_CLASSE,CNPJ_FUNDO_CLASSE,ID_SUBCLASSE,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
391050,CLASSES - FIF,48.931.854/0001-18,<NA>,2025-12-01,33093580.0,1.665717,32774418.93,318412.33,89555.77,751


In [359]:
fundo , classe , subclase , df = busca_cadastro_cvm()


In [360]:
df[df['CNPJ_Classe']=='48.931.854/0001-18']

,ID_Registro_Classe,ID_Subclasse,Codigo_CVM,Data_Constituicao,Data_Inicio,Denominacao_Social,Situacao,Data_Inicio_Situacao,Forma_Condominio,Exclusivo,...,Data_Inicio_Exercicio_Social,Data_Fim_Exercicio_Social,Patrimonio_Liquido_fundo,Data_Patrimonio_Liquido_fundo,Diretor,CNPJ_Administrador,Administrador,Tipo_Pessoa_Gestor,CPF_CNPJ_Gestor,Gestor
78612,17745.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2025-10-01,2026-09-30,8181576.04,2024-10-10,GUSTAVO COTTA PIERSANTI,5.928125e+13,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,2.102353e+13,AVANTGARDE ASSET MANAGEMENT GESTAO DE RECURSOS...


In [361]:
df[df['CNPJ_Fundo']=='48.931.854/0001-18']

,ID_Registro_Classe,ID_Subclasse,Codigo_CVM,Data_Constituicao,Data_Inicio,Denominacao_Social,Situacao,Data_Inicio_Situacao,Forma_Condominio,Exclusivo,...,Data_Inicio_Exercicio_Social,Data_Fim_Exercicio_Social,Patrimonio_Liquido_fundo,Data_Patrimonio_Liquido_fundo,Diretor,CNPJ_Administrador,Administrador,Tipo_Pessoa_Gestor,CPF_CNPJ_Gestor,Gestor
78612,17745.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2025-10-01,2026-09-30,8181576.04,2024-10-10,GUSTAVO COTTA PIERSANTI,5.928125e+13,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,2.102353e+13,AVANTGARDE ASSET MANAGEMENT GESTAO DE RECURSOS...


In [362]:
classe[classe['CNPJ_Classe']=='48.931.854/0001-18']

,ID_Registro_Fundo,ID_Registro_Classe,CNPJ_Classe,Codigo_CVM,Data_Registro,Data_Constituicao,Data_Inicio,Tipo_Classe,Denominacao_Social,Situacao,...,Exclusivo,Publico_Alvo,Patrimonio_Liquido,Data_Patrimonio_Liquido,CNPJ_Auditor,Auditor,CNPJ_Custodiante,Custodiante,CNPJ_Controlador,Controlador
26282,74158,17745,48.931.854/0001-18,68357,2024-07-18,2022-12-14,2024-07-18,Classes de Cotas de Fundos FIF,AVANTGARDE MULTIFATORES ABSOLUTO FUNDO DE INVE...,Em Funcionamento Normal,...,N,Público Geral,42071625.13,2026-01-15,6.136694e+13,ERNST & YOUNG AUDITORES INDEPENDENTES S/S LTDA.,3.030629e+13,BANCO BTG PACTUAL S/A,5.928125e+13,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM


In [363]:
df.columns

Index(['ID_Registro_Classe', 'ID_Subclasse', 'Codigo_CVM', 'Data_Constituicao',
       'Data_Inicio', 'Denominacao_Social', 'Situacao', 'Data_Inicio_Situacao',
       'Forma_Condominio', 'Exclusivo', 'Publico_Alvo', 'ID_Registro_Fundo',
       'CNPJ_Classe', 'Codigo_CVM_classe', 'Data_Registro',
       'Data_Constituicao_classe', 'Data_Inicio_classe', 'Tipo_Classe',
       'Denominacao_Social_classe', 'Situacao_classe',
       'Data_Inicio_Situacao_classe', 'Classificacao', 'Indicador_Desempenho',
       'Classe_Cotas', 'Classificacao_Anbima', 'Tributacao_Longo_Prazo',
       'Entidade_Investimento', 'Permitido_Aplicacao_CemPorCento_Exterior',
       'Classe_ESG', 'Forma_Condominio_classe', 'Exclusivo_classe',
       'Publico_Alvo_classe', 'Patrimonio_Liquido', 'Data_Patrimonio_Liquido',
       'CNPJ_Auditor', 'Auditor', 'CNPJ_Custodiante', 'Custodiante',
       'CNPJ_Controlador', 'Controlador', 'CNPJ_Fundo', 'Codigo_CVM_fundo',
       'Data_Registro_fundo', 'Data_Constituicao_fundo',

In [364]:
fundo.columns

Index(['ID_Registro_Fundo', 'CNPJ_Fundo', 'Codigo_CVM', 'Data_Registro',
       'Data_Constituicao', 'Tipo_Fundo', 'Denominacao_Social',
       'Data_Cancelamento', 'Situacao', 'Data_Inicio_Situacao',
       'Data_Adaptacao_RCVM175', 'Data_Inicio_Exercicio_Social',
       'Data_Fim_Exercicio_Social', 'Patrimonio_Liquido',
       'Data_Patrimonio_Liquido', 'Diretor', 'CNPJ_Administrador',
       'Administrador', 'Tipo_Pessoa_Gestor', 'CPF_CNPJ_Gestor', 'Gestor'],
      dtype='object')

In [365]:
classe.columns

Index(['ID_Registro_Fundo', 'ID_Registro_Classe', 'CNPJ_Classe', 'Codigo_CVM',
       'Data_Registro', 'Data_Constituicao', 'Data_Inicio', 'Tipo_Classe',
       'Denominacao_Social', 'Situacao', 'Data_Inicio_Situacao',
       'Classificacao', 'Indicador_Desempenho', 'Classe_Cotas',
       'Classificacao_Anbima', 'Tributacao_Longo_Prazo',
       'Entidade_Investimento', 'Permitido_Aplicacao_CemPorCento_Exterior',
       'Classe_ESG', 'Forma_Condominio', 'Exclusivo', 'Publico_Alvo',
       'Patrimonio_Liquido', 'Data_Patrimonio_Liquido', 'CNPJ_Auditor',
       'Auditor', 'CNPJ_Custodiante', 'Custodiante', 'CNPJ_Controlador',
       'Controlador'],
      dtype='object')

In [366]:
subclase.columns

Index(['ID_Registro_Classe', 'ID_Subclasse', 'Codigo_CVM', 'Data_Constituicao',
       'Data_Inicio', 'Denominacao_Social', 'Situacao', 'Data_Inicio_Situacao',
       'Forma_Condominio', 'Exclusivo', 'Publico_Alvo'],
      dtype='object')

filtros

In [367]:
use_cols_cadastro = [
    'ID_Registro_Classe', 'ID_Subclasse', 'ID_Registro_Fundo', 
    'CNPJ_Classe', 'CNPJ_Fundo',
    # 'Codigo_CVM', 'Codigo_CVM_classe',  'Codigo_CVM_fundo',
    'Denominacao_Social', 'Denominacao_Social_classe', 'Denominacao_Social_fundo',
    # 'Tipo_Classe', 'Tipo_Fundo','Classificacao', 'Classificacao_Anbima', 
    ]

In [368]:
cad = df[
     # (df['Situacao_classe'] == 'Em Funcionamento Normal') & 
     # (df['Publico_Alvo'] != 'Profissional') & 
     (df['Exclusivo'] == 'N') & 
     # (df['Forma_Condominio'] == 'Aberto') & 
     (df['Entidade_Investimento'] != 'N')  & 
     (df['Situacao'] == 'Em Funcionamento Normal')  
    ][use_cols_cadastro]

In [369]:
# lista de cnpjs de interesse
cnpjs = cad['CNPJ_Fundo'].drop_duplicates().to_list()

In [370]:
len(cnpjs)

1241

rendimento no mes

In [371]:
fundos_filtrados = diario[
    (diario['CNPJ_FUNDO_CLASSE'].isin(cnpjs)) &
    (diario['NR_COTST'] > 1000)
    ].sort_values(by='DT_COMPTC')[['CNPJ_FUNDO_CLASSE' , 'ID_SUBCLASSE' , 'DT_COMPTC' , 'VL_QUOTA' , 'NR_COTST']].reset_index(drop=True)

In [372]:
len(fundos_filtrados)

535

In [373]:
len(fundos_filtrados.CNPJ_FUNDO_CLASSE.drop_duplicates())

24

In [374]:
# 1. Verificar se há duplicatas
duplicatas = fundos_filtrados.groupby(['DT_COMPTC', 'CNPJ_FUNDO_CLASSE']).size()
print("Registros duplicados:")
print(duplicatas[duplicatas > 1])

Registros duplicados:
DT_COMPTC   CNPJ_FUNDO_CLASSE 
2025-12-01  39.586.858/0001-15    2
2025-12-02  39.586.858/0001-15    2
2025-12-03  39.586.858/0001-15    2
2025-12-04  39.586.858/0001-15    2
2025-12-05  39.586.858/0001-15    2
2025-12-08  39.586.858/0001-15    2
2025-12-09  39.586.858/0001-15    2
2025-12-10  39.586.858/0001-15    2
2025-12-11  39.586.858/0001-15    2
2025-12-12  39.586.858/0001-15    2
2025-12-15  39.586.858/0001-15    2
2025-12-16  39.586.858/0001-15    2
2025-12-17  39.586.858/0001-15    2
2025-12-18  39.586.858/0001-15    2
2025-12-19  39.586.858/0001-15    2
2025-12-22  39.586.858/0001-15    2
2025-12-23  39.586.858/0001-15    2
2025-12-24  39.586.858/0001-15    2
2025-12-26  39.586.858/0001-15    2
2025-12-29  39.586.858/0001-15    2
2025-12-30  39.586.858/0001-15    2
2025-12-31  39.586.858/0001-15    2
dtype: int64


In [375]:
df[df['CNPJ_Fundo']=='39.586.858/0001-15'][[ 'ID_Subclasse',  'Denominacao_Social', 
              'Denominacao_Social_classe', 
      'Classificacao',  'Classificacao_Anbima', 
                               'Denominacao_Social_fundo', 
       ]]

,ID_Subclasse,Denominacao_Social,Denominacao_Social_classe,Classificacao,Classificacao_Anbima,Denominacao_Social_fundo
69365,7EV3M1743800974,SUBCLASSE IV KINEA IPCA DINÂMICO II FUNDO DE I...,KINEA IPCA DINÂMICO II FUNDO DE INVESTIMENTO F...,Renda Fixa,Renda Fixa Duração Livre Crédito Livre,KINEA IPCA DINÂMICO II FUNDO DE INVESTIMENTO F...
69366,DZV4X1753974840,SUBCLASSE V DO KINEA IPCA DINÂMICO II FUNDO DE...,KINEA IPCA DINÂMICO II FUNDO DE INVESTIMENTO F...,Renda Fixa,Renda Fixa Duração Livre Crédito Livre,KINEA IPCA DINÂMICO II FUNDO DE INVESTIMENTO F...
69367,KB0OK1743799805,SUBCLASSE I DO KINEA IPCA DINÂMICO II FUNDO DE...,KINEA IPCA DINÂMICO II FUNDO DE INVESTIMENTO F...,Renda Fixa,Renda Fixa Duração Livre Crédito Livre,KINEA IPCA DINÂMICO II FUNDO DE INVESTIMENTO F...
69368,M0UTC1743800258,SUBCLASSE II DO KINEA IPCA DINÂMICO II FUNDO D...,KINEA IPCA DINÂMICO II FUNDO DE INVESTIMENTO F...,Renda Fixa,Renda Fixa Duração Livre Crédito Livre,KINEA IPCA DINÂMICO II FUNDO DE INVESTIMENTO F...
69369,YPI761743800738,SUBCLASSE III DO KINEA IPCA DINÂMICO II FUNDO ...,KINEA IPCA DINÂMICO II FUNDO DE INVESTIMENTO F...,Renda Fixa,Renda Fixa Duração Livre Crédito Livre,KINEA IPCA DINÂMICO II FUNDO DE INVESTIMENTO F...


In [376]:
fundos_filtrados[fundos_filtrados['CNPJ_FUNDO_CLASSE']=='39.586.858/0001-15']

,CNPJ_FUNDO_CLASSE,ID_SUBCLASSE,DT_COMPTC,VL_QUOTA,NR_COTST
16,39.586.858/0001-15,KB0OK1743799805,2025-12-01,1.677592,7678
19,39.586.858/0001-15,M0UTC1743800258,2025-12-01,1.642755,8423
33,39.586.858/0001-15,KB0OK1743799805,2025-12-02,1.680921,7660
35,39.586.858/0001-15,M0UTC1743800258,2025-12-02,1.646015,8404
56,39.586.858/0001-15,KB0OK1743799805,2025-12-03,1.682403,7647
57,39.586.858/0001-15,M0UTC1743800258,2025-12-03,1.647466,8389
77,39.586.858/0001-15,KB0OK1743799805,2025-12-04,1.683292,7631
78,39.586.858/0001-15,M0UTC1743800258,2025-12-04,1.648337,8381
99,39.586.858/0001-15,KB0OK1743799805,2025-12-05,1.671321,7623
111,39.586.858/0001-15,M0UTC1743800258,2025-12-05,1.636614,8369


In [377]:
def limpa_dados_cvm(df):
    # Ordenar por data para garantir a sequência temporal
    # df = df.sort_values(['CNPJ_FUNDO_CLASSE', 'DT_COMPTC'])
    
    # Remover duplicatas exatas de CNPJ e Data 
    # (Mantendo a primeira ocorrência, ou filtrando por TP_FUNDO_CLASSE se preferir)
    df = df.drop_duplicates(subset=['CNPJ_FUNDO_CLASSE',  'DT_COMPTC'], keep='first')
    
    return df

In [378]:
fundos_tratados = fundos_filtrados.drop_duplicates(subset=['CNPJ_FUNDO_CLASSE',  'DT_COMPTC'], keep='first').copy()
duplicatas = fundos_tratados.groupby(['DT_COMPTC', 'CNPJ_FUNDO_CLASSE']).size()
print("Registros duplicados:")
print(duplicatas[duplicatas > 1])

Registros duplicados:
Series([], dtype: int64)


In [379]:
fundos_tratados[fundos_tratados['DT_COMPTC']=='2025-12-12']

,CNPJ_FUNDO_CLASSE,ID_SUBCLASSE,DT_COMPTC,VL_QUOTA,NR_COTST
216,34.687.633/0001-31,XELXE1746019072,2025-12-12,1.720906,2953
217,34.793.170/0001-92,BNAX91750170440,2025-12-12,1.803925,1388
218,59.960.677/0001-14,SH7DQ1742324712,2025-12-12,1.079799,1225
219,34.626.199/0001-80,UMGNS1765399777,2025-12-12,1.768700,1194
220,35.609.382/0001-30,ZN9EI1750172960,2025-12-12,1.970607,4042
221,33.736.845/0001-07,FI3N41752603729,2025-12-12,9.175605,36296
222,61.543.507/0001-86,P0UDD1751405646,2025-12-12,1.037086,2085
223,00.888.897/0001-31,MZMRC1747322915,2025-12-12,66.377844,12709
224,24.140.256/0001-62,CEMTO1749244596,2025-12-12,2.543742,2215
225,39.586.858/0001-15,M0UTC1743800258,2025-12-12,1.643496,8306


In [380]:
# Usa a média caso haja duplicatas (ou 'last', 'first', 'sum', etc.)
quota = fundos_tratados.pivot_table(
    index='DT_COMPTC', 
    columns='CNPJ_FUNDO_CLASSE',
    values='VL_QUOTA',  # Especifique apenas a coluna que precisa
    aggfunc='last'  # ou 'mean', 'first', etc.
)

In [381]:
fundos_tratados.shape

(513, 5)

In [382]:
quota.shape

(22, 24)

In [383]:
cotas_normalizadas = quota / quota.iloc[0]
rentabilidade = (((cotas_normalizadas.iloc[-1] - 1)*100).round(2)).to_frame(name='rendeu (%)').reset_index()

In [384]:
rentabilidade.sort_values(by='rendeu (%)',ascending=False)

,CNPJ_FUNDO_CLASSE,rendeu (%)
11,33.736.845/0001-07,4.84
18,41.672.453/0001-41,4.68
4,14.438.229/0001-17,3.85
8,29.726.133/0001-21,2.09
16,36.440.431/0001-16,1.35
9,29.762.315/0001-58,1.26
20,59.960.677/0001-14,1.21
10,32.990.051/0001-02,1.19
1,07.900.255/0001-50,1.17
15,35.609.382/0001-30,1.17


In [385]:
cotas_normalizadas

CNPJ_FUNDO_CLASSE,00.888.897/0001-31,07.900.255/0001-50,09.412.694/0001-49,11.952.873/0001-10,14.438.229/0001-17,21.624.757/0001-26,24.140.256/0001-62,27.945.264/0001-74,29.726.133/0001-21,29.762.315/0001-58,...,34.793.170/0001-92,35.609.382/0001-30,36.440.431/0001-16,39.586.858/0001-15,41.672.453/0001-41,57.553.569/0001-00,59.960.677/0001-14,60.018.029/0001-22,61.333.973/0001-37,61.543.507/0001-86
DT_COMPTC,,,,,,,,,,,,,,,,,,,,,
2025-12-01,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2025-12-02,1.000887,1.000112,NaN,1.000376,1.006328,1.002427,1.000512,1.000941,1.006062,1.003205,...,1.002230,1.000152,1.001355,1.001985,0.988414,1.001859,1.000549,1.001194,1.000551,1.001154
2025-12-03,1.001449,0.999988,NaN,1.000753,1.014397,1.004010,0.998819,1.001514,1.011348,1.005249,...,1.003221,1.000154,1.004226,1.002868,0.986209,1.003504,1.001098,1.002116,1.001103,1.002017
2025-12-04,1.001946,1.000483,NaN,1.001709,1.024359,1.005831,1.001398,1.002288,1.010842,1.007535,...,1.003973,1.000585,1.001325,1.003398,0.985119,1.005974,1.001634,1.004535,1.001655,1.003123
2025-12-05,1.001908,1.000880,NaN,1.002658,1.006183,1.000828,0.992719,1.000930,0.994160,1.000890,...,1.000530,1.001374,0.999314,0.996262,1.008837,1.002126,1.002159,0.997569,1.002207,1.001901
2025-12-08,1.002028,1.001275,NaN,1.002378,1.007129,1.001168,0.992905,1.001436,0.992176,1.001182,...,1.001310,1.002835,0.999500,0.997251,1.002635,1.001681,1.002709,0.996892,1.002760,1.001751
2025-12-09,1.003028,1.002098,NaN,1.003041,1.006808,1.001990,0.992930,1.002013,0.990848,1.002205,...,1.000245,1.003689,0.999674,0.977220,1.009135,0.999566,1.003277,0.994931,1.003312,1.001591
2025-12-10,1.003053,1.002459,NaN,1.003431,1.020391,1.001862,0.993312,1.002051,0.991218,1.001861,...,0.999810,1.004022,0.999717,0.975907,1.019585,0.999132,1.003843,0.993507,1.003866,1.001311
2025-12-11,1.003823,1.003051,NaN,1.004440,1.022751,1.003985,0.993875,1.002665,0.999722,1.004657,...,1.001463,1.004643,1.001592,0.997491,1.018693,1.001530,1.004406,0.992087,1.004419,1.001133


In [386]:
rentabilidade.sort_values(by='rendeu (%)',ascending=False).dropna()

,CNPJ_FUNDO_CLASSE,rendeu (%)
11,33.736.845/0001-07,4.84
18,41.672.453/0001-41,4.68
4,14.438.229/0001-17,3.85
8,29.726.133/0001-21,2.09
16,36.440.431/0001-16,1.35
9,29.762.315/0001-58,1.26
20,59.960.677/0001-14,1.21
10,32.990.051/0001-02,1.19
1,07.900.255/0001-50,1.17
15,35.609.382/0001-30,1.17


In [ ]:
base = pd.merge(fundos_filtrados, dados_cadastro.drop(columns='TP_FUNDO'), how = "left",
                      left_on = ["CNPJ_FUNDO"], right_on = ["CNPJ_FUNDO"])
base_final = pd.merge(base,rentabilidade, how = "left",
                      left_on = ["CNPJ_FUNDO"], right_on = ["CNPJ_FUNDO"])

# base_final

NameError: name 'dados_cadastro' is not defined

In [ ]:
base_final.columns

Index(['TP_FUNDO', 'CNPJ_FUNDO', 'DT_COMPTC', 'VL_TOTAL', 'VL_QUOTA',
       'VL_PATRIM_LIQ', 'CAPTC_DIA', 'RESG_DIA', 'NR_COTST', 'DENOM_SOCIAL',
       'SIT', 'CLASSE', 'RENTAB_FUNDO', 'CONDOM', 'TRIB_LPRAZO',
       'PUBLICO_ALVO', 'TAXA_PERFM', 'INF_TAXA_PERFM', 'TAXA_ADM',
       'INF_TAXA_ADM', 'CLASSE_ANBIMA', 'rendeu (%)'],
      dtype='object')

Rendimento de um fundo específico

In [ ]:
busca_fundo = base_final[base_final['DENOM_SOCIAL'].str.contains("DYNAMO", na = False)]

from IPython.display import display, HTML
display(HTML(busca_fundo[['CNPJ_FUNDO' ,  'DENOM_SOCIAL']].drop_duplicates().to_html(index=False)))

In [ ]:
cnpj = "73.232.530/0001-39"
FI = busca_fundo[(busca_fundo['CNPJ_FUNDO'] == cnpj)][['DENOM_SOCIAL', 'VL_PATRIM_LIQ','rendeu (%)']].iloc[-1].values

print(FI[0])
print(f'Patrimônio R$ {(FI[1]/1000000).round(2)} MM') 
print(f'Retorno no mês {FI[2]}%')

DYNAMO COUGAR FUNDO DE INVESTIMENTO EM COTAS DE FUNDO DE INVESTIMENTO EM AÇÕES
Patrimônio R$ 6250.62 MM
Retorno no mês 3.32%


#### melhores e piores fundos

In [ ]:
base_final['CLASSE'].unique()

array(['Renda Fixa', 'Ações', 'Multimercado', 'Cambial', nan, 'FMP-FGTS',
       'FIDC', 'FIC FIDC', 'FIP Multi', 'FIDC-NP'], dtype=object)

In [ ]:
# mostra os melhores que satifazem as restrições
minimo_cotistas = 5000
top = 50
classe = 'Multimercado'
'''
[nan, 'Fundo de Renda Fixa', 'Fundo de Ações', 'Fundo Multimercado',
       'Fundo Cambial', 'FMP-FGTS', 'FIDC', 'FIDC-NP', 'FIC FIDC',
       'FICFIDC-NP', 'FIDCFIAGRO', 'FII', 'FII-FIAGRO', 'FIP', 'FIP EE',
       'FIP Multi', 'FIP CS', 'FIP-FIAGRO', 'FIC FIP', 'FIP IE',
       'FUNCINE']
'''       

filtro = (base_final['SIT'] == 'EM FUNCIONAMENTO NORMAL')&\
              (base_final['NR_COTST'] >= minimo_cotistas)&\
              (base_final['CLASSE'] == classe)

infos = ['VL_PATRIM_LIQ', 'NR_COTST', 'DENOM_SOCIAL',
        'PUBLICO_ALVO', 'TAXA_PERFM', 'TAXA_ADM',
        'CLASSE_ANBIMA', 'rendeu']

infos = ['DENOM_SOCIAL','TAXA_PERFM', 'TAXA_ADM','CLASSE_ANBIMA', 'rendeu (%)']

melhores

In [ ]:
# base_final[filtro].sort_values(by='rendeu',ascending=False)[infos]
display(HTML(base_final[filtro].sort_values(by='rendeu (%)',ascending=False)[infos].head(top).to_html(index=False)))

DENOM_SOCIAL,TAXA_PERFM,TAXA_ADM,CLASSE_ANBIMA,rendeu (%)
HASHDEX CRYPTO SELECTION FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO,20.0,2.00,Multimercados Estrat. Específica,46.49
HASHDEX BITCOIN FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO,0.0,0.70,Multimercados Estrat. Específica,45.73
HASHDEX BITCOIN FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO,0.0,0.70,Multimercados Estrat. Específica,45.73
HASHDEX 100 NASDAQ CRYPTO INDEX FUNDO DE INVESTIMENTO MULTIMERCADO,0.0,0.70,Multimercados Estrat. Específica,44.52
HASHDEX 100 NASDAQ CRYPTO INDEX FUNDO DE INVESTIMENTO MULTIMERCADO,0.0,0.70,Multimercados Estrat. Específica,44.52
TREND CRIPTO DÓLAR FUNDO DE INVESTIMENTO MULTIMERCADO,NaN,NaN,Multimercados Estrat. Específica,43.91
TREND CRIPTO DÓLAR FUNDO DE INVESTIMENTO MULTIMERCADO,NaN,NaN,Multimercados Estrat. Específica,43.91
EMPIRICUS COIN CRIPTO FUNDO DE INVESTIMENTO MULTIMERCADO,NaN,NaN,Multimercados Estrat. Específica,41.90
EMPIRICUS COIN CRIPTO FUNDO DE INVESTIMENTO MULTIMERCADO,NaN,NaN,Multimercados Estrat. Específica,41.90
EMPIRICUS CRIPTOMOEDAS FI EM COTAS DE FUNDO DE INVESTIMENTO MULTIMERCADO INVESTIMENTO NO EXTERIOR,0.0,0.05,Multimercados Invest. no Exterior,35.57


piores

In [ ]:
display(HTML(base_final[filtro].sort_values(by='rendeu (%)',ascending=True)[infos].head(top).to_html(index=False)))

DENOM_SOCIAL,TAXA_PERFM,TAXA_ADM,CLASSE_ANBIMA,rendeu (%)
TREND CANNABIS FUNDO DE INVESTIMENTO MULTIMERCADO,0.0,0.70,Multimercados Livre,-12.42
TREND CANNABIS FUNDO DE INVESTIMENTO MULTIMERCADO,0.0,0.70,Multimercados Livre,-12.42
BTG PACTUAL TACTICAL BONDS FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,2.20,Multimercados Invest. no Exterior,-7.99
BTG PACTUAL TACTICAL BONDS FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,2.20,Multimercados Invest. no Exterior,-7.99
BTG PACTUAL TACTICAL BONDS PRÉ II FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,2.20,Multimercados Invest. no Exterior,-7.13
BTG PACTUAL TACTICAL BONDS PRÉ II FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,2.20,Multimercados Invest. no Exterior,-7.13
VISTA HEDGE FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO,20.0,2.00,Multimercados Livre,-4.76
VISTA HEDGE FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO,20.0,2.00,Multimercados Livre,-4.76
GAP ABSOLUTO ADVISORY FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO,NaN,NaN,Multimercados Livre,-3.54
GAP ABSOLUTO ADVISORY FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO,NaN,NaN,Multimercados Livre,-3.54


rank por palavra chave no nome

In [ ]:
palavra = 'quant'
classe = 'Multimercado'

filtro = (base_final['SIT'] == 'EM FUNCIONAMENTO NORMAL')&\
    (base_final['CLASSE'] == classe)&\
    (base_final['DENOM_SOCIAL'].str.contains(palavra.upper()))

display(HTML(base_final[filtro].sort_values(by='rendeu (%)',ascending=False)[infos].to_html(index=False)))

DENOM_SOCIAL,TAXA_PERFM,TAXA_ADM,CLASSE_ANBIMA,rendeu (%)
MANAGER GLOBAL QUANT LONG-BIASED EQUITIES FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO IE,0.0,0.80,Multimercados Invest. no Exterior,2.67
MANAGER GLOBAL QUANT LONG-BIASED EQUITIES FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO IE,0.0,0.80,Multimercados Invest. no Exterior,2.67
QUANTITAS FUNDO DE INVESTIMENTO MULTIMERCADO MALDIVAS LONG SHORT,20.0,1.80,Multimercados L/S - Direcional,2.28
QUANTITAS FUNDO DE INVESTIMENTO MULTIMERCADO MALDIVAS LONG SHORT,20.0,1.80,Multimercados L/S - Direcional,2.28
COPACABANA QUANTITATIVO FUNDO DE INVESTIMENTO MULTIMERCADO,0.0,0.99,Multimercados Livre,1.11
COPACABANA QUANTITATIVO FUNDO DE INVESTIMENTO MULTIMERCADO,0.0,0.99,Multimercados Livre,1.11
BB MULTIMERCADO LP QUANTITATIVO PRIVATE FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,0.0,1.10,Multimercados Livre,1.10
BB MULTIMERCADO LP QUANTITATIVO PRIVATE FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,0.0,1.10,Multimercados Livre,1.10
BB MULTIMERCADO QUANTITATIVO LP FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,0.0,1.50,Multimercados Livre,1.08
BB MULTIMERCADO QUANTITATIVO LP FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,0.0,1.50,Multimercados Livre,1.08
